## Setup

Add any imports needed here

Mount Google Drive

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip -q install datasets
!pip -q install transformers
!pip -q install peft
!pip -q install -U bitsandbytes
!pip -q install rouge


import numpy as np
from datasets import load_dataset
import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive
from bs4 import BeautifulSoup

drive.mount('/content/drive')

#%cd /content/drive/MyDrive/CS6220 Folder #add direct access to the folder if you get an error in this cell
import os
os.chdir("/content/drive/MyDrive/CS6220 Folder")

import helpers
import importlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.8 MB/s eta 0:00:00
Mounted at /content/drive


## **IBM Python Dataset**

## Dataset

We use a different code dataset which we then partition into artificial "repositories" using PyTorch dataloader

In [ ]:
## TODO: Diana
## Download Dataset
%cd /content/drive/.shortcut-targets-by-id/1ZKIbzAJY4RXvlu64yVQt4OckCxyNqRJ6/CS6220 Folder

/content


The next approach uses the Scripts_Project_CodeNet_Python800 folder. It contains .py scripts. Each script has multiple chunks of code (combined from the previous structure, where in each folder we had multiple scripts). The delimiter between chunk and chunk is a string of hashtags. (#####################################################)

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import random


class CustomDataset(Dataset):

    def __init__(self, folder_path, partition, delimiter="#", chunk_size=100): #initializes the dataset with the folder path, delimiter, and chunk size
        self.folder_path = folder_path
        self.delimiter = delimiter
        self.chunk_size = chunk_size

        self.script_files = [f for f in os.listdir(folder_path) if f.endswith(".py")] # Get a list of all Python scripts in the specified folder with a ".py" extension
        if partition == 1:
          self.script_files = self.script_files[:len(self.script_files) // 2]
        else:
          self.script_files = self.script_files[len(self.script_files) // 2+1:]

        self.data = []

        for idx in range(len(self.script_files)):
          script_file_path = os.path.join(self.folder_path, self.script_files[idx])
          with open(script_file_path, 'r', encoding='utf-8') as file:
              code_samples = file.read().split(self.delimiter)
              code_samples = [i for i in code_samples if len(i) > 5]
              self.data.extend(code_samples)
        print(f'Code samples in partition {partition}: {len(self.data)}')

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx): # method to get a specific item from the dataset (returns a chunk of code)
        return self.data[idx]


In [ ]:
folder_path = "/content/drive/.shortcut-targets-by-id/1ZKIbzAJY4RXvlu64yVQt4OckCxyNqRJ6/CS6220 Folder/Scripts_Project_CodeNet_Python800"
delimiter = "#####################################################"
chunk_size = 1
batch_size = 4

# Create instances of the dataset with specified file indices
dataset_1 = CustomDataset(folder_path, 1, delimiter, chunk_size)
dataset_2 = CustomDataset(folder_path, 2, delimiter, chunk_size)

# Create DataLoader instances for each dataset
dataloader_1 = DataLoader(dataset_1, batch_size=4, shuffle=True)
dataloader_2 = DataLoader(dataset_2, batch_size=4, shuffle=True)

client_dataloaders = [dataloader_1, dataloader_2]

total = 0
for batch in dataloader_1:
    total += 1
print(total)

#for batch in data_loader:
#    print(batch)

Code samples in partition 1: 120137
Code samples in partition 2: 119825
30035


## Pretrained model

This model [name] from Hugging Face is pretrained on code.

In [ ]:
## TODO: Jun
## Download model
# Load the CodeParrot small model and tokenizer

from helpers.train import generic_training_runner
from transformers import BitsAndBytesConfig
from peft import LoraConfig

model_name = "codeparrot/codeparrot-small"
run_title = "codeparrotsm"


SAVE_PATH = f"./ronit_ibm_outputs/checkpoints/{run_title}"
LOSSES_PATH = f"./ronit_ibm_outputs/logs/losses/{run_title}"
TIMES_PATH = f"./ronit_ibm_outputs/logs/times/{run_title}_elapsed_time"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

model_info = {
    'model_name': model_name,
    'tokenizer': tokenizer,
    'client_dataloaders': client_dataloaders,
    'quant_config':
        BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        ),
    'lora_config':
        LoraConfig(
            r=8,
            target_modules=["qkv_proj", "out_proj"],
            lora_alpha=32,
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM",
        ),
}

training_args = {
    'clients': 2,
    'data_per_client': 1e3,
    'MAX_LENGTH': 1024,
    'conduct_logging': True,
    'EPOCHS': 1,
    'lr': 4e-4
}


In [ ]:
def generate_response(prompt, model, tokenizer):
  input_ids = tokenizer(prompt, return_tensors="pt").to(device).input_ids

  generated_ids = model.generate(input_ids, max_length=128)
  return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generate_response("def hello_world():", model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
    return "Hello World!"

def hello_world_with_no_args():
    return "Hello World!"

def hello_world_with_args():
    return "Hello World!"

def hello_world_with_kwargs():
    return "Hello World!"

def hello_world_with_kwargs_and_kwargs():
    return "Hello World!"

def hello_world_with_kwargs_and_kwargs_and_kwargs():
    return "Hello World!"

def hello_world_with_kwargs_and_kwargs_and_kwargs


## Partition Code [outdated]

In [ ]:
# ## Partition Dataset

# import random

# # Assuming you have 'n_partitions' as the number of partitions you want
# n_partitions = 5  # for example, change this based on your requirement

# # Shuffle the dataset
# random.shuffle(custom_dataset.script_files)

# # Calculate the size of each partition
# partition_size = len(custom_dataset) // n_partitions

# # Create partitions
# partitions = [custom_dataset.script_files[i * partition_size:(i + 1) * partition_size] for i in range(n_partitions)]

# # Now create a DataLoader for each partition
# dataloaders = []
# for partition in partitions:
#     # Creating a new CustomDataset instance for each partition
#     partition_dataset = CustomDataset(folder_path, delimiter, chunk_size)
#     partition_dataset.script_files = partition  # replace the script_files with the partition

#     # Create a DataLoader for the partitioned dataset
#     partition_dataloader = DataLoader(partition_dataset, batch_size=1, shuffle=True)
#     dataloaders.append(partition_dataloader)

# # Now 'dataloaders' contains separate DataLoader for each partition

# import hashlib
# from itertools import islice

# def hash_code(data):
#     # Create a hash for the given data
#     return hashlib.sha256(str(data).encode()).hexdigest()

# # Iterate through each DataLoader
# for i, each_data_loader in enumerate(dataloaders):
#     print(f"\nDataLoader {i + 1}:")

#     # Iterate through the first 5 batches of each DataLoader
#     for j, batch in enumerate(islice(each_data_loader, 5), 1):
#         # Compute the hash of the batch
#         batch_hash = hash_code(batch)

#         print(f"  Batch {j} Hash: {batch_hash}")





DataLoader 1:


TypeError: ignored

In [ ]:
# # Validation Script

# # Check for equal size
# partition_sizes = [len(partition) for partition in partitions]
# print("Sizes of each partition:", partition_sizes)

# # Check if sizes are approximately equal
# # Note: They might not be exactly equal if the total size is not perfectly divisible by n_partitions
# if all(abs(size - partition_size) <= 1 for size in partition_sizes):
#     print("All partitions are of approximately equal size.")
# else:
#     print("Warning: Partitions sizes vary more than expected.")

# # Consistency Check
# # Example: Checking if each partition still contains script files and not something else
# for i, partition_dataset in enumerate(dataloaders):
#     is_consistent = all(isinstance(script, str) and script.endswith('.py') for script in partition_dataset.dataset.script_files)
#     print(f"Partition {i} consistency: {'Consistent' if is_consistent else 'Inconsistent'}")

# # Note: Replace the consistency check as per your criteria


# def hash_code(code_snippet):
#     # Using SHA-256 for hashing
#     return hashlib.sha256(code_snippet.encode()).hexdigest()

# # Check for duplicates within each partition
# for i, partition in enumerate(partitions):
#     hash_set = set()
#     duplicates = set()
#     for snippet in partition:
#         snippet_hash = hash_code(snippet)
#         if snippet_hash in hash_set:
#             duplicates.add(snippet_hash)
#         else:
#             hash_set.add(snippet_hash)

#     if duplicates:
#         print(f"Partition {i} has duplicates: {duplicates}")
#     else:
#         print(f"Partition {i} has no duplicates.")

# # Check for duplicates across partitions
# global_hash_set = set()
# cross_partition_duplicates = set()
# for partition in partitions:
#     for snippet in partition:
#         snippet_hash = hash_code(snippet)
#         if snippet_hash in global_hash_set:
#             cross_partition_duplicates.add(snippet_hash)
#         else:
#             global_hash_set.add(snippet_hash)

# if cross_partition_duplicates:
#     print(f"Cross-partition duplicates found: {cross_partition_duplicates}")
# else:
#     print("No cross-partition duplicates found.")




Sizes of each partition: [160, 160, 160, 160, 160]
All partitions are of approximately equal size.
Partition 0 consistency: Consistent
Partition 1 consistency: Consistent
Partition 2 consistency: Consistent
Partition 3 consistency: Consistent
Partition 4 consistency: Consistent
Partition 0 has no duplicates.
Partition 1 has no duplicates.
Partition 2 has no duplicates.
Partition 3 has no duplicates.
Partition 4 has no duplicates.
No cross-partition duplicates found.


## **Deduplication** [outdated]

In [ ]:
#dataset = load_dataset("codeparrot/codeparrot-clean")

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
#########not enough RAM -- try process in chunks ??????????
path = '/content/drive/MyDrive/CS6220 Folder/codeparrot-clean/extracted/'

import json
import pandas as pd

all_code_content = []

for i in range(1, 55):
    file_name = f'file-{i:012}.json'
    full_path = f'{path}{file_name}'
    try:
        with open(full_path, 'r') as file:
            for line in file:
                try:
                    data = json.loads(line)
                    code_content = data.get('content', '')
                    if code_content:
                        all_code_content.append({'content': code_content})
                except json.JSONDecodeError as e:
                    print(f"JSON decode error in file {file_name}, line: {line}")
    except FileNotFoundError as e:
        print(f"File not found: {e}")

df = pd.DataFrame(all_code_content)



KeyboardInterrupt: ignored

## **Big Code/ The-Stack** [outdated]

In [ ]:
!pip install huggingface-cli
!huggingface-cli login


In [ ]:
from  datasets  import  load_dataset

# full dataset (3TB of data)
#ds = load_dataset("bigcode/the-stack", split="train")

# specific language (e.g. Dockerfiles)
ds = load_dataset("bigcode/the-stack", data_dir="data/python", split="train")

# dataset streaming (will only download the data as needed)
#ds = load_dataset("bigcode/the-stack", streaming=True, split="train")
#for sample in iter(ds): print(sample["content"])


##run script

In [ ]:
generic_training_runner(
    SAVE_PATH,
    LOSSES_PATH,
    TIMES_PATH,
    model_info,
    training_args,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Device Type
cuda
Beginning training iteration
Number of batches: 30035
Starting training


1it [00:00,  6.01it/s]

Step: 0, Loss: 5.7070


5it [00:00,  6.20it/s]

Step: 3, Loss: 4.3516


...

11939it [30:57,  6.52it/s]

Step: 11937, Loss: 6.6094


11942it [30:57,  6.56it/s]

Step: 11940, Loss: 6.4531


11944it [30:57,  6.78it/s]